In [ ]:
from cifar10_classifier import CIFAR10Classifier
from cifar10_model import CIFAR10_torch
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_cls = CIFAR10Classifier(
    name="torch_1024_5",
    input_shape=input_shape,
    num_classes=num_classes,
    hidden_layers=[1024,512,256,128,64],
    dropout_rate=[0.3,0.3,0.3,0.3,0.3],
    lr=LR,
    device=device
    )
model_cls.build_model()
model_cls.compile()
model_cls.summary()